In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [2]:
column_names = ['Page Popularity', 'Page Checkins', 'Page Talking About', 'Page Category', 
           'Derived_1', 'Derived_2', 'Derived_3', 'Derived_4', 'Derived_5', 
           'Derived_6', 'Derived_7', 'Derived_8', 'Derived_9', 'Derived_10',
           'Derived_11', 'Derived_12', 'Derived_13', 'Derived_14', 'Derived_15', 
           'Derived_16', 'Derived_17', 'Derived_18', 'Derived_19', 'Derived_20', 
           'Derived_21', 'Derived_22', 'Derived_23', 'Derived_24', 'Derived_25', 
           'CC1', 'CC2', 'CC3', 'CC4', 'CC5',
           'Base Time', 'Post Length', 'Post Share Count', 'Post Promotion Status', 'Local Hours',
           'Sunday Published', 'Monday Published', 'Tuesday Published', 'Wednesday Published', 'Thursday Published', 'Friday Published', 'Saturday Published', 
           'Sunday Base', 'Monday Base', 'Tuesday Base', 'Wednesday Base', 'Thursday Base', 'Friday Base', 'Saturday Base', 
           'target']

In [3]:
data = pd.read_csv('facebook_comment.csv', header=None, names=column_names)

In [4]:
data.head()

,Page Popularity,Page Checkins,Page Talking About,Page Category,Derived_1,Derived_2,Derived_3,Derived_4,Derived_5,Derived_6,...,Friday Published,Saturday Published,Sunday Base,Monday Base,Tuesday Base,Wednesday Base,Thursday Base,Friday Base,Saturday Base,target
0,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,0,1,0
1,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,1,0,0
2,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,0,0,0,0,0,1,0
3,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,1,0,0,0,0,0,0
4,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,1,0,0,0,0


## Normalization

In [5]:
from sklearn.preprocessing import StandardScaler

Заменим все NaN на 0.

In [6]:
if data.isna().any().sum() != 0:
    data.fillna(value=0, inplace=True)

Удалим все столбцы, в которых у всех элементов одинаковые значения.

In [7]:
drop_columns = []

In [8]:
print('Data Size: ', data.shape)

for column in data.columns:
    size = len(np.unique(data[column]))
    if size == 1:
        drop_columns += [column]

print('Drop Columns: ', drop_columns)

Data Size:  (40949, 54)
Drop Columns:  ['Post Promotion Status']


In [9]:
data = data.drop(columns=drop_columns)

In [10]:
data.dtypes

Page Popularity          int64
Page Checkins            int64
Page Talking About       int64
Page Category            int64
Derived_1              float64
Derived_2              float64
Derived_3              float64
Derived_4              float64
Derived_5              float64
Derived_6              float64
Derived_7              float64
Derived_8              float64
Derived_9              float64
Derived_10             float64
Derived_11             float64
Derived_12             float64
Derived_13             float64
Derived_14             float64
Derived_15             float64
Derived_16             float64
Derived_17             float64
Derived_18             float64
Derived_19             float64
Derived_20             float64
Derived_21             float64
Derived_22             float64
Derived_23             float64
Derived_24             float64
Derived_25             float64
CC1                      int64
CC2                      int64
CC3                      int64
CC4     

Нормализуем каждую из фич

In [11]:
old_columns = data.drop(columns=['Local Hours', 'target']).columns
new_columns = []

In [12]:
for column in old_columns:
    new_column = column + ' Norm'
    new_columns += [new_column]
    data[new_column] = StandardScaler().fit_transform(np.array(data[column]).reshape(-1,1))

In [13]:
data.shape

(40949, 104)

Преобразуем время

In [14]:
print(np.unique(data['Local Hours']))

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]


In [15]:
def make_harmonic_features(value, period=24):
    value *= 2 * np.pi / period
    return np.cos(value), np.sin(value)

data['Harmonic Cos Local Hours'] = make_harmonic_features(data['Local Hours'], 24)[0]
data['Harmonic Sin Local Hours'] = make_harmonic_features(data['Local Hours'], 24)[1]

old_columns += ['Local Hours']
new_columns += ['Harmonic Cos Local Hours', 'Harmonic Sin Local Hours']

In [16]:
norm_data = data[new_columns]
norm_data.head()

,Page Popularity Norm,Page Checkins Norm,Page Talking About Norm,Page Category Norm,Derived_1 Norm,Derived_2 Norm,Derived_3 Norm,Derived_4 Norm,Derived_5 Norm,Derived_6 Norm,...,Saturday Published Norm,Sunday Base Norm,Monday Base Norm,Tuesday Base Norm,Wednesday Base Norm,Thursday Base Norm,Friday Base Norm,Saturday Base Norm,Harmonic Cos Local Hours,Harmonic Sin Local Hours
0,-0.100037,-0.227075,-0.399678,-1.165633,-0.076435,0.730167,-0.511079,-0.495224,0.037159,-0.021827,...,-0.398309,-0.406216,-0.390827,-0.399501,-0.417774,-0.421477,-0.409962,2.429384,1.0,0.997253
1,-0.100037,-0.227075,-0.399678,-1.165633,-0.076435,0.730167,-0.511079,-0.495224,0.037159,-0.021827,...,-0.398309,-0.406216,-0.390827,-0.399501,-0.417774,-0.421477,2.439248,-0.411627,1.0,0.997253
2,-0.100037,-0.227075,-0.399678,-1.165633,-0.076435,0.730167,-0.511079,-0.495224,0.037159,-0.021827,...,-0.398309,-0.406216,-0.390827,-0.399501,-0.417774,-0.421477,-0.409962,2.429384,1.0,0.997253
3,-0.100037,-0.227075,-0.399678,-1.165633,-0.076435,0.730167,-0.511079,-0.495224,0.037159,-0.021827,...,-0.398309,-0.406216,2.558674,-0.399501,-0.417774,-0.421477,-0.409962,-0.411627,1.0,0.997253
4,-0.100037,-0.227075,-0.399678,-1.165633,-0.076435,0.730167,-0.511079,-0.495224,0.037159,-0.021827,...,-0.398309,-0.406216,-0.390827,-0.399501,2.393637,-0.421477,-0.409962,-0.411627,1.0,0.997253


## Regression

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

In [18]:
def RMSE(y_test, y_pred):
    return np.sqrt(MSE(y_test, y_pred))

In [19]:
X = np.array(norm_data)
y = np.array(data['target'])

In [20]:
model_lr = GradientBoostingRegressor()
model_coefs = []
model_stats = []

kf = KFold(n_splits=5)
kf.get_n_splits(X)
for train_index, test_index in kf.split(X):
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    
    model_lr.fit(X_train, y_train)
    y_pred = model_lr.predict(X_test)
    
    model_coefs += [model_lr.feature_importances_]
    model_stats += [[np.average(y_test-y_pred), np.std(y_test-y_pred), MSE(y_test, y_pred), r2_score(y_test, y_pred)]]   

In [21]:
pd.DataFrame(model_coefs)

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
0,0.006218,0.004964,0.001575,0.000710,0.000000,0.003951,0.001090,0.004427,0.009253,0.000229,...,0.000000,0.000000,0.0,0.000192,0.009748,0.000046,0.000000,0.000000,0.000000,0.000278
1,0.007466,0.007275,0.005996,0.002019,0.004350,0.000369,0.006802,0.007995,0.001830,0.001169,...,0.000000,0.000367,0.0,0.000173,0.000023,0.000003,0.000000,0.000000,0.000000,0.000072
2,0.001552,0.007250,0.005486,0.000000,0.003722,0.000516,0.005566,0.003564,0.005739,0.000058,...,0.000006,0.000000,0.0,0.000151,0.003805,0.000000,0.000000,0.000073,0.000000,0.000005
3,0.004556,0.005215,0.002214,0.006753,0.000000,0.001559,0.000223,0.000381,0.003668,0.000018,...,0.000000,0.000000,0.0,0.000000,0.000949,0.000000,0.000002,0.000000,0.000000,0.000094
4,0.003717,0.006673,0.008915,0.001561,0.000051,0.003746,0.001067,0.001380,0.005626,0.000000,...,0.000000,0.000000,0.0,0.000188,0.004963,0.000000,0.000075,0.000000,0.000032,0.000035


In [22]:
pd.DataFrame(model_stats, columns = ['E', 'STD', 'MSE', 'R2'])

,E,STD,MSE,R2
0,0.669864,21.378630,457.494517,0.709210
1,0.541932,23.572981,555.979135,0.542520
2,0.178620,20.789034,432.215832,0.704613
3,-0.995953,25.651049,658.968242,0.483034
4,-0.785235,17.502888,306.967697,0.598753
